In [29]:
from langchain.chat_models import ChatOpenAI
from RAG.VectorBase import VectorStore
from RAG.utils import ReadFiles
from RAG.LLM import OpenAIChat
from RAG.Embeddings import BgeEmbedding
from typing import Dict, List, Optional, Tuple, Union

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ChatMessage
)

In [41]:
PROMPT_TEMPLATE = dict(
    RAG_PROMPT_TEMPALTE="""结合以上下文来回答用户的问题。
        问题: {question}
        可参考的上下文：
        ···
        {context}
        ···
        如果给定的上下文无法让你做出回答，请回答数据库中没有这个内容，不要臆想推测，使用中文回答。
        回答:""",
)


class MyChatOpenAI():
    def __init__(self, path: str = '', model: str = "Qwen2.5-7B-Instruct",
                 api_key: str = "YOUR API KEY", base_url: str = "http://localhost:6006/v1",
                 temperature: float = 0.1) -> None:
        super().__init__()
        self.model = model
        self.client = ChatOpenAI(
            api_key=api_key,
            base_url=base_url,
            temperature=temperature,
            model_name=model
        )

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        history.append(
            HumanMessage(content=PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content))
        )
        response = self.client(messages=history)
        return response.content


In [43]:
# 没有保存数据库
docs = ReadFiles('./data').get_content(max_token_len=600, cover_content=150)  # 获得data目录下的所有文件内容并分割
vector = VectorStore(docs)
embedding = BgeEmbedding(path='D:/MyProject/TorchProject/NLP/model_hub/bge-small-zh-v1.5')  # 创建EmbeddingModel
vector.get_vector(EmbeddingModel=embedding)
vector.persist(path='storage')  # 将向量和文档内容保存到storage目录下，下次再用就可以直接加载本地的数据库

question = '什么是git？'

content = vector.query(question, EmbeddingModel=embedding, k=1)[0]

chat = MyChatOpenAI()
print(chat.chat(question, [], content))

Calculating embeddings: 100%|█████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 128.71it/s]


Git是一种分布式版本控制系统，用于跟踪文件的变化并支持协作开发。它可以让每个开发者拥有整个代码库的历史记录，无需依赖中央服务器，并且分支和合并操作非常高效，适合大型项目的开发。


In [45]:
vector = VectorStore()

vector.load_vector('./storage')  # 加载本地的数据库

embedding = BgeEmbedding(path='D:/MyProject/TorchProject/NLP/model_hub/bge-small-zh-v1.5')

question = 'git的分支原理？'

content = vector.query(question, EmbeddingModel=embedding, k=1)[0]

chat = MyChatOpenAI()
print(chat.chat(question, [], content))

Git的分支原理主要是用来隔离开发工作的。每个分支都是一个独立的开发环境，互不影响。分支可以很方便地被创建和合并，因此许多开发者使用分支来进行特性开发、修复bug或者尝试新想法。

在Git中，几乎所有操作都是本地执行的，包括创建和切换分支。你可以使用以下命令来操作分支：

- 创建新分支：`git branch <分支名>`
- 切换到新分支：`git checkout <分支名>`
- 创建并立即切换到新分支：`git checkout -b <分支名>`
- 合并指定分支到当前分支：`git merge <分支名>`

合理的分支策略可以帮助团队更有效地协作。例如，GitFlow定义了一个围绕项目发布的分支模型，包括功能分支、发布分支、维护分支等。而GitHubFlow则更加简单灵活，适合持续交付的项目。在GitHubFlow中，master分支通常是稳定的，并且随时可以部署。所有新的开发都在基于master的特性分支上进行，一旦完成就可以合并回master。

如果在合并分支时遇到冲突，Git会提示你需要手动解决这些冲突。解决冲突后，可以继续完成合并操作。
